In [1]:
import pandas as pd
import requests
from datetime import datetime


In [2]:
def remove_multiindex(df:pd.DataFrame):
	return df.droplevel(level=0,axis=1)

In [3]:
# link for extract html data
def getdata(url):
    r = requests.get(url)
    return r.text



In [32]:
full_df_list = pd.DataFrame(columns=[ 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
		   '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
		   'BLK', 'TOV', 'PF', 'PTS', 'Age', 'W', 'L', 'MOV',
		   'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%',])


for year in range(1980,datetime.now().year + 1):

	htmldata = getdata(f"https://www.basketball-reference.com/leagues/NBA_{year}.html")
	df_list = pd.read_html(htmldata)

	if year <= 2015:
		per_game_stat = df_list[3].copy()

		adv_stat = df_list[8].copy()
	else:
		per_game_stat = df_list[4].copy()

		adv_stat = df_list[10].copy()

	new = pd.merge(per_game_stat,adv_stat.droplevel(level=0,axis=1),on='Team')

	new = new[[ 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
		   '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
		   'BLK', 'TOV', 'PF', 'PTS', 'Age', 'W', 'L', 'MOV',
		   'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%',]].sort_values('W',ascending=False).reset_index(drop=True)

	new['Year'] = year

	new.loc[new['Team'] == 'League Average','Team'] = f'League Average {year}'

	new.to_csv(f'team_stats/{year}_tm_stats.csv')
	full_df_list = full_df_list.append(new)

	print(year,'Complete')


full_df_list.to_csv('team_stats/all.csv')

1980 Complete
1981 Complete
1982 Complete
1983 Complete
1984 Complete
1985 Complete
1986 Complete
1987 Complete
1988 Complete
1989 Complete
1990 Complete
1991 Complete
1992 Complete
1993 Complete
1994 Complete
1995 Complete
1996 Complete
1997 Complete
1998 Complete
1999 Complete
2000 Complete
2001 Complete
2002 Complete
2003 Complete
2004 Complete
2005 Complete
2006 Complete
2007 Complete
2008 Complete
2009 Complete
2010 Complete
2011 Complete
2012 Complete
2013 Complete
2014 Complete
2015 Complete
2016 Complete
2017 Complete
2018 Complete
2019 Complete
2020 Complete
2021 Complete
2022 Complete


In [34]:
full_df_list.to_csv('../Data/Team Data/Combined/league_stats.csv')